In [ ]:
import pickle
from pathlib import Path

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [ ]:
X_train = pd.read_csv("./data/processed/X_train.csv")
X_test = pd.read_csv("./data/processed/X_test.csv")
y_train = pd.read_csv("./data/processed/y_train.csv")
y_test = pd.read_csv("./data/processed/y_test.csv")

In [ ]:
models_params = {
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=1000, random_state=42),
        "params": {
            "C": [0.01, 0.1, 1, 10, 100],
            "solver": ["liblinear", "lbfgs"],
            "penalty": ["l2"],
        },
    },

    "Random Forest": {
        "model": RandomForestClassifier(random_state=42, n_jobs=-1),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [10, 20, 30],
            "min_samples_split": [2, 5],
            "min_samples_leaf": [1, 2],
            "max_features": ["sqrt"],
        },
    },

    "XGBoost": {
        "model": XGBClassifier(random_state=42, eval_metric="logloss", n_jobs=-1),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [3, 5, 7],
            "learning_rate": [0.01, 0.1, 0.3],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
        },
    },
}

In [ ]:
results = {}
for name, mp in models_params.items():
    grid_search = GridSearchCV(
        estimator=mp["model"],
        param_grid=mp["params"],
        cv=5,
        scoring="accuracy",
        n_jobs=-1,
        verbose=1,
    )

    grid_search.fit(X_train, y_train)

    results[name] = {
        "best_model": grid_search.best_estimator_,
        "best_params": grid_search.best_params_,
        "test_score": grid_search.score(X_test, y_test),
    }

In [ ]:
comparison = pd.DataFrame({
    "model": list(results.keys()),
    "test_score": [results[m]["test_score"] for m in results],
}).sort_values("test_score", ascending=False)

print(comparison.to_string(index=False))

In [ ]:
best_model_name = comparison.iloc[0]["model"]
best_model = results[best_model_name]["best_model"]

print(f"Best Model: {best_model_name}")
print(f"Test Accuracy: {results[best_model_name]['test_score']:.4f}")
print(f"Best Hyperparameters: {results[best_model_name]['best_params']}")

In [ ]:
with Path.open("./model/best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)